# Collaboration and Competition



In this notebook we go about training the agent with the MADDPG algorithm. To this end we import the required packages and load the enviromenent. Make sure that the Tennis app is in the local directory.

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from training import train
import matplotlib.pyplot as plt
import torch
from model import Actor
from IPython.display import Image


In [2]:
env = UnityEnvironment(file_name="Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


# Training the Model

Here you can adjust hyperparameters of the training. It has the capability to do MADDPG or MAD4PG. 


In [3]:
param={    
    'n_step':1, #how many time steps we take before we make our TD estimate
    'n_episodes':10000, #maximum episodes to run (code will stop if average score is above 30 before this number)
    'noise':True,# controls whether Ornstein–Uhlenbeck noise is added to each parameter of the actor
    'print':False,
    'agentparam':
    {
        'device':'cpu',
        'action_size' : 2, 
        'state_size' : 24,
        'seed' : 10,
        'update_times':1, #how many times we learn at every step

        'ddpgtype':'singleval', #distribution type: 'catdist' for categorical distribution (MAD4PG) or "singleval" for classic MADDPG
        'atoms' : 51, #atoms for D4PG


        'catparam':
        {
            'vmin' : 0, #minimum q category
            'vmax' : 1 #maximum q category
        },

        'gamma' : 0.999, #discount factor : how much future rewards are discounted
        'lr_actor' : 1e-4, #actor network learning rate
        'lr_critic' : 1e-3, #critic network learning rate
        'l2weights' : 0, #weight decay in adam optimizer, equivalent to l2 regularization
        'tau' : 1e-3, #controls how quickly the target networks are updated
        'explore_factor' :0.1, #controls how much noise is added to each action parameter
        'batch_size' : 512,#how many experiences sampled in each batch

        'replay_param' :
        {
            'buffer_size': int(1e6), #max length of buffer
        },
    
        'OUparam':
        {
            'sigma': 0.5, #width of noise distribution
            'mu':0, #Noise will be centered around mu
            'theta':0.15 #how large a step we take towards the mean
        },
        
        'criticparam':
        {
            'layer_units':[300,150] #List items create layers with n number of neurons, add item for additional layer
        }, 
        
        'actorparam':
        {
            'layer_units':[200,100]  #List items create layers with n number of neurons, add item for additional layer
        }
        

    }
      }

In [4]:
scores,avgscores=train(env,param)

Episode 100	Average Score: 0.00
Episode 200	Average Score: 0.01
Episode 300	Average Score: 0.01
Episode 400	Average Score: 0.03
Episode 500	Average Score: 0.02
Episode 600	Average Score: 0.00
Episode 700	Average Score: 0.00
Episode 800	Average Score: 0.01
Episode 900	Average Score: 0.02
Episode 1000	Average Score: 0.04
Episode 1100	Average Score: 0.03
Episode 1200	Average Score: 0.07
Episode 1300	Average Score: 0.11
Episode 1400	Average Score: 0.12
Episode 1500	Average Score: 0.09
Episode 1600	Average Score: 0.11
Episode 1700	Average Score: 0.13
Episode 1800	Average Score: 0.15
Episode 1900	Average Score: 0.18
Episode 2000	Average Score: 0.19
Episode 2100	Average Score: 0.23
Episode 2200	Average Score: 0.98
Episode 2300	Average Score: 2.78


AttributeError: 'MultiAgent' object has no attribute 'actionestimator_local'

The environment is considered solved at 1 and thus here it is solved. By running the following cell our results can be seen in tensorboard.

In [5]:
!tensorboard --logdir=runs

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


The graph of score vs episode for the training session above is also presented below.

In [6]:
Image(url='AverageScoreMADDPG.png')  


# Testing the agent

Here we load and test our agent

In [4]:
TrainedActor0=Actor(24,2,10,param['agentparam']['actorparam'])
TrainedActor0.load_state_dict(torch.load('checkpoint_actor0.pth'))
TrainedActor0.eval()

Actor(
  (hlayers): Sequential(
    (0): Linear(in_features=24, out_features=200, bias=True)
    (1): SELU()
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): SELU()
  )
  (out): Linear(in_features=100, out_features=2, bias=True)
  (outlayer): Sequential(
    (0): Linear(in_features=100, out_features=2, bias=True)
    (1): Tanh()
  )
)

In [5]:
TrainedActor1=Actor(24,2,10,param['agentparam']['actorparam'])
TrainedActor1.load_state_dict(torch.load('checkpoint_actor1.pth'))
TrainedActor1.eval()

Actor(
  (hlayers): Sequential(
    (0): Linear(in_features=24, out_features=200, bias=True)
    (1): SELU()
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): SELU()
  )
  (out): Linear(in_features=100, out_features=2, bias=True)
  (outlayer): Sequential(
    (0): Linear(in_features=100, out_features=2, bias=True)
    (1): Tanh()
  )
)

In [6]:
actors=[TrainedActor0,TrainedActor1]
actions=np.zeros((2,2))
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [11]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
num_agents=20                                          # change to appropriate number for your enviroment
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    states=torch.from_numpy(states).float()
    for i,state in enumerate(states):
        actions[i,:] = actors[i](state).detach().numpy()    # select an action (for each agent)
        actions[i,:] = np.clip(actions[i,:], -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += np.max(env_info.rewards)                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))


Total score (averaged over agents) this episode: 4.6000000685453415


In [15]:
Image(url='trainedTennis.gif')  


This is a gif of our agent getting the score of 4.6

In [3]:
env.close()

# Future Improvements

In the future we could try a few things. I think implementing starting with a greater noise and exploration and then introducing a decay hyperparameter would help us find the right model parameters quickly and then converge well once we have reached found the first good area to search for neural network weights. In MADDPG it took us till around episode 1800 to find this area and this could help us find it sooner and solve our system with less training time. 

Hyperparameters could also be searched with a grid search, given enough training time to find the hyperparameters that could achieve a certain score with the least amount of training.

Prioritized replay might also help with our sittuation at the beggining by selecting the most important experiences. It seemed once the model started getting enough of the right experiences it rapidly learned and so prioritized replay in particular has good potential too improve our model